In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# first import names.csv file and turn into a list of names
import csv

with open('names.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the first row
    with open('names.txt', 'w') as txtfile:
        for i, row in enumerate(csvreader):
            if i < 1000:  # for the first 1000 rows
                name1 = row[1]
                name2 = row[2]
                txtfile.write(name1 + '\n')
                txtfile.write(name2 + '\n') 

In [4]:
# turn all letters to lowercase. this is important because 'A' and 'a' are different characters for the model
with open('names.txt', 'r') as f:
    names = f.read()

with open('names.txt', 'w') as f:
    f.write(names.lower())

In [5]:
# we can now read the names.txt file
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(words[:10])

2000
['olivia', 'liam', 'emma', 'noah', 'charlotte', 'oliver', 'amelia', 'elijah', 'ava', 'james']


In [6]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [8]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([13643, 3]), torch.int64, torch.Size([13643]), torch.int64)

In [9]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([10920, 3]) torch.Size([10920])
torch.Size([1354, 3]) torch.Size([1354])
torch.Size([1369, 3]) torch.Size([1369])


In [10]:
C = torch.randn((27, 2))
emb = C[X]
emb.shape
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.2793, -0.9999, -0.9999,  ...,  0.9983,  0.9881,  0.9229],
        [ 0.5234, -0.9945, -0.9999,  ...,  0.9991,  0.8084,  0.9395],
        [-0.9436, -0.9987, -0.9958,  ...,  0.9963, -0.7242, -0.8997],
        ...,
        [ 0.1191, -0.2369, -0.9955,  ..., -0.3018, -0.5357,  0.5439],
        [-0.3670, -0.9926, -0.9958,  ...,  0.9874, -0.4554, -0.8972],
        [-0.9645, -0.9396, -0.9629,  ...,  0.1279, -0.9532, -0.9756]])

In [11]:
h.shape

torch.Size([13643, 100])

In [12]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
logits = h @ W2 + b2
logits.shape

torch.Size([13643, 27])

In [13]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
prob.shape

torch.Size([13643, 27])

In [14]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [13643]

In [15]:
# new optimized code

In [16]:
Xtr.shape, Ytr.shape # dataset size

(torch.Size([10920, 3]), torch.Size([10920]))

In [17]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [18]:
sum(p.nelement() for p in parameters) # number of parameters in total

11897

In [19]:
for p in parameters:
  p.requires_grad = True

In [20]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [21]:
lri = []
lossi = []
stepi = []

In [22]:
for i in range(200000):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,))
  
  # forward pass
  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  #print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

#print(loss.item())

In [ ]:
plt.plot(stepi, lossi)
